In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
all_sites = pd.read_csv('../data/all_sites.csv')

In [7]:
all_sites4 = all_sites[9300:12400]

In [4]:
# Select model
nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(nlp_model)
model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

# Classifier
classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer)

In [5]:
def sentiment_analysis_bert_base_multilingual_uncased(review):
    global_stars = []
    n = 1900
    
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]    
    global_stars = (classifier(review_splitted))
    
    return global_stars


In [30]:
def apply_sentiment_model(df):
    try:
        df['stars'] = df['text'].apply(lambda x: sentiment_analysis_bert_base_multilingual_uncased(x))
        
    except RuntimeError:
        print(df['game'])
        df['stars'] = df['text'].apply(lambda x: np.nan)
        
    return df

In [31]:
apply_sentiment_model(all_sites4)

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,site,url_link,author,game,company,genre,platform,text,score,stars
0,meristation,https://as.com/meristation/2008/12/22/analisis...,Javier de Pascual,Metal Gear Solid Mobile,Konami,"Acción, Tactical",NGE,"Por alguna extraña razón, cuando el jugón trad...",8.5,"[{'label': '3 stars', 'score': 0.4448424875736..."
1,meristation,https://as.com/meristation/2012/12/19/analisis...,Pablo González,THE DENPA MEN: They Came By Wave,None,RPG,3DS,Nintendo 3DS ha apostado desde su primer día e...,7.2,"[{'label': '3 stars', 'score': 0.4176394641399..."
2,meristation,https://as.com/meristation/2019/09/03/analisis...,David Arroyo,Telling Lies,Annapurna Interactive,Aventura,PC PS4 XBO NSW,Hablar de Telling Lies es un poco complicado. ...,8.0,"[{'label': '3 stars', 'score': 0.4085363745689..."
3,meristation,https://as.com/meristation/2014/10/06/analisis...,Salva Fernàndez,Titanfall: IMC Rising,None,"Acción, First Person Shooter",PC 360 XBO,Titanfall recibe el último contenido descargab...,7.0,"[{'label': '3 stars', 'score': 0.4825307130813..."
4,meristation,https://as.com/meristation/2014/04/16/analisis...,None,Moebius: Empire Rising,None,Aventura Gráfica,PC OSX IPH IPD AND,"“Aquellos que no pueden recordar el pasado, es...",6.8,"[{'label': '4 stars', 'score': 0.4670569002628..."


In [ ]:
def stars_mean_to_score(review):
    
    points = []

    for classification in review:

        #Mean of fragment
        grade = int(classification['label'].split(' ')[0])
        points.append(grade)  
    score = round(np.mean(points), 2)

    return(score)   

In [ ]:
all_sites4['stars_mean'] = all_sites4['stars'].apply(lambda x: stars_mean_to_score(x))

In [ ]:
def score_labels(score):
    scorelabel = f"'{round(score / 2)} starts'"
    return scorelabel

In [ ]:
all_sites4['score_label'] = all_sites4['score'].apply(lambda x: score_labels(x))

In [ ]:
def score_half(score):
    return score / 2

In [ ]:
all_sites4['score_adj'] = all_sites4['score'].apply(lambda x: score_half(x))

In [ ]:
all_sites4.to_csv('../data/scored_texts4.csv', index=False)